# Deep learning volatility

In [2]:
%config Completer.use_jedi=False
# import gzip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import csv
import pandas as pd

import scipy

In [3]:
import os
os.chdir('Data')
# os.getcwd()


In [3]:
df= np.loadtxt("impliedVols_total.txt")

xx = df[:, :6]
yy = df[:, 6:]

# # total = np.zeros((len(yy), 6+strikes_dim*maturities_dim))
# # total[:,:6] = xx
# # for i in range(maturities_dim):
# #     print(total[:, 6+i*strikes_dim:((i+1)*strikes_dim - 1)].shape,yy[:, i*(strikes_dim-1):((i+1)*(strikes_dim - 1))].shape)
# #     total[:, 6+i*strikes_dim:(6+(i+1)*strikes_dim - 1)] = yy[:, i*(strikes_dim-1):((i+1)*(strikes_dim-1))]
# #     total[:, 6+(i+1)*strikes_dim-1] = df_[:, i]
# print(len(df123), len(df123[0]))
# np.savetxt("parameters_impliedVols_T7K15.txt", df123, fmt='%.4f')       
len(df), len(df[0])


UnicodeDecodeError: 'gbk' codec can't decode byte 0xef in position 129: illegal multibyte sequence

In [ ]:
maturities = np.array([0.05, 0.1, 0.2, 0.3, 0.5, 0.75, 1.0])# Time to expiry in years
# strikes = np.array([80, 85, 90, 95, 97, 99, 100, 101, 103, 105, 110, 115, 120, 122])
strikes = np.array([82., 85., 90., 95., 98., 100., 102., 105., 110., 115., 122.])
# strikes = np.array([5.25, 5.50, 5.75, 6.00, 6.25])
logmoneyness = np.log(strikes/100.)
strikes_dim = len(strikes)
maturities_dim = len(maturities)
strikes_dim, maturities_dim

In [ ]:
for i in range(6):
    print(np.min(df[:, i]),np.max(df[:, i]))

In [ ]:
np.where(yy == yy.min())
# yy[0][12]
yy.min(axis=0)
# np.where(yy == yy.min())

In [ ]:
#remove invalid paramters
invalid = np.where(yy == yy.min())[0]
xx = np.delete(xx, (invalid), axis=0)
yy = np.delete(yy, (invalid), axis=0)
xx.shape, yy.shape

In [ ]:
for i in range(6):
    print(np.min(xx[:, i]),np.max(xx[:, i]))

# Step 1: Data Generation


### **Inputs: parameters in the rough quadratic Heston model** 
####   1.  $\alpha\in(0.5, 0.7);$
####   2. $\lambda\in(0.5, 1.5);$
####   3. $a\in(0.1, 0.75);$
####   4. $b\in(0.05,0.5);$
####   5. $c\in(0.0001, 0.01);$
####   6. $Z_0\in(0.05, 0.3).$  

    
### **Outputs:**
#### Implied volatility surface (14x9=126 points)

## Scaling:

### Model Parameters $\theta\to scale(\theta)\in[0,1]$
$$scale(\theta^i)=\frac{(\theta^i-\theta^i_{min})}{\theta^i_{max}-\theta^i_{min}},\quad i \in |\Theta|$$

### Implied volatilities
$$scale(\sigma_{BS}^{i,j})=\frac{\sigma_{BS}^{i,j}-\mathbb{E}[\sigma_{BS}^{i,j}]}{std(\sigma_{BS}^{i,j})},\quad i\in\text{Maturities},\; j\in \text{Strikes}$$

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    xx, yy, test_size=0.15, random_state=42)
# from sklearn.preprocessing import StandardScaler, MinMaxScaler

# scale = StandardScaler()
# y_train_transform = scale.fit_transform(y_train)
# y_test_transform = scale.transform(y_test)

$\alpha\in[0.505, 0.7],\; \lambda\in[0.5, 1.5],\; a\in[0.1, 0.75],\; b\in[0.05, 0.5],\; c\in[0.0001, 0.01],\; Z_0\in[0.05, 0.3]$

In [ ]:
ub=np.max(xx, axis=0)
lb=np.min(xx, axis=0)
print(ub, lb)
def myscale(x):
    res=np.zeros(6)
    for i in range(6):
        res[i]=(x[i] - (ub[i] + lb[i])*0.5) * 2 / (ub[i] - lb[i])
        
    return res
def myinverse(x):
    res=np.zeros(6)
    for i in range(6):
        res[i]=x[i]*(ub[i] - lb[i]) *0.5 + (ub[i] + lb[i])*0.5
        
    return res

In [ ]:
x_train_transform = np.array([myscale(x) for x in X_train])
x_test_transform = np.array([myscale(x) for x in X_test])
x_test_transform.shape
# np.max(x_train_transform, axis=0)

# Step 2. Create the Neural Network

In [ ]:
import keras
from keras.layers import Activation
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects
keras.backend.set_floatx('float64')


input1 = keras.layers.Input(shape=(6,))
x1 = keras.layers.Dense(30,activation = 'elu')(input1)
x2=keras.layers.Dense(30,activation = 'elu')(x1) 
x3=keras.layers.Dense(30,activation = 'elu')(x2) 


x4=keras.layers.Dense(77,activation = 'linear')(x3)


modelGEN = keras.models.Model(inputs=input1, outputs=x4)
modelGEN.summary()

# Step 3. Fit the Neural Network (No need to run the code, parameters are stored on a .h5 file)

In [ ]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))
modelGEN.compile(loss = root_mean_squared_error, optimizer = "adam")



In [ ]:
modelGEN.fit(x_train_transform, y_train, 
             batch_size=32,
#              validation_split = 0.2,
             validation_data = (x_test_transform, y_test),
             epochs = 200, verbose = True,shuffle=1)#,callbacks=[earlystop])

### Store/Load optimal NN parameteres

In [ ]:
modelGEN.save_weights('qrHestonNNWeights.h5')

In [ ]:
modelGEN.load_weights('qrHestonNNWeights.h5')

In [ ]:
NNParameters=[]
for i in range(1,len(modelGEN.layers)):
    NNParameters.append(modelGEN.layers[i].get_weights())
# np.savetxt("nnParameters.txt", np.asarray(NNParameters))
# NNParameters

# Step 4. Optimisation
### (Numpy Implementation of the Neural Network tends to be faster than Keras)

Here the *elu* function writes:
$$elu(x) = \begin{cases}e^{x}-1,\; x<0\\
x,\; x\geq 0.
\end{cases}$$

In [ ]:
NumLayers=3
def elu(x):
    #Careful function ovewrites x
    ind=(x<0)
    x[ind]=np.exp(x[ind])-1
    return x
def eluPrime(y):
    # we make a deep copy of input x
    x=np.copy(y)
    ind=(x<0)
    x[ind]=np.exp(x[ind])
    x[~ind]=1
    return x
def NeuralNetwork(x):
    input1=x
    for i in range(NumLayers):
        input1=np.dot(input1,NNParameters[i][0])+NNParameters[i][1]
        #Elu activation
        input1=elu(input1)
    #The output layer is linnear
    i+=1
    return np.dot(input1,NNParameters[i][0])+NNParameters[i][1]
def NeuralNetworkGradient(x):
    input1=x
    #Identity Matrix represents Jacobian with respect to initial parameters
    grad=np.eye(6)
    #Propagate the gradient via chain rule
    for i in range(NumLayers):
        input1=(np.dot(input1,NNParameters[i][0])+NNParameters[i][1])
        grad=(np.einsum('ij,jk->ik', grad, NNParameters[i][0]))
        #Elu activation
        grad*=eluPrime(input1)
        input1=elu(input1)
    #input1.append(np.dot(input1[i],NNParameters[i+1][0])+NNParameters[i+1][1])
#     grad=np.einsum('ij,jk->ik',grad,NNParameters[i+1][0])
    grad=np.dot(grad,NNParameters[i+1][0])
    #grad stores all intermediate Jacobians, however only the last one is used here as output
    return grad

# Compute Implied vol relative errors on Test set

In [ ]:
NeuralNetwork(X_sample[0]) 

In [ ]:
##### AVERAGE VALUES #######
X_sample = x_test_transform
y_sample = y_test

# prediction=[scale.inverse_transform(modelGEN.predict(X_sample[i].reshape(1,6))[0]) for i in range(len(X_sample))]
# prediction=[modelGEN.predict(X_sample[i].reshape(1,6))[0] for i in range(len(X_sample))]
prediction=[NeuralNetwork(X_sample[i]) for i in range(len(X_sample))]


plt.figure(1,figsize=(14,4))
ax=plt.subplot(1,3,1)
err = np.mean(100*np.abs((y_sample-prediction)/y_sample),axis = 0)
RMSE = np.sqrt(np.mean(np.power(y_sample-prediction, 2), axis=1)) ### compute the RMSE of the calibrated smile surface

plt.title("Average relative error",fontsize=15,y=1.04)
plt.imshow(err.reshape(maturities_dim,strikes_dim))
plt.colorbar(format=mtick.PercentFormatter())

ax.set_xticks(np.linspace(0,len(strikes)-1,len(strikes)))
ax.set_xticklabels(strikes)
ax.set_yticks(np.linspace(0,len(maturities)-1,len(maturities)))
ax.set_yticklabels(maturities)
plt.xlabel("Strike",fontsize=15,labelpad=5)
plt.ylabel("Maturity",fontsize=15,labelpad=5)

ax=plt.subplot(1,3,2)
err = 100*np.std(np.abs((y_sample-prediction)/y_sample),axis = 0)
plt.title("Std relative error",fontsize=15,y=1.04)
plt.imshow(err.reshape(maturities_dim,strikes_dim))
plt.colorbar(format=mtick.PercentFormatter())
ax.set_xticks(np.linspace(0,len(strikes)-1,len(strikes)))
ax.set_xticklabels(strikes)
ax.set_yticks(np.linspace(0,len(maturities)-1,len(maturities)))
ax.set_yticklabels(maturities)
plt.xlabel("Strike",fontsize=15,labelpad=5)
plt.ylabel("Maturity",fontsize=15,labelpad=5)

ax=plt.subplot(1,3,3)
err = 100*np.max(np.abs((y_sample-prediction)/y_sample),axis = 0)
plt.title("Maximum relative error",fontsize=15,y=1.04)
plt.imshow(err.reshape(maturities_dim,strikes_dim))
plt.colorbar(format=mtick.PercentFormatter())
ax.set_xticks(np.linspace(0,len(strikes)-1,len(strikes)))
ax.set_xticklabels(strikes)
ax.set_yticks(np.linspace(0,len(maturities)-1,len(maturities)))
ax.set_yticklabels(maturities)
plt.xlabel("Strike",fontsize=15,labelpad=5)
plt.ylabel("Maturity",fontsize=15,labelpad=5)
plt.tight_layout()
# plt.savefig('qrHestonNNErrors.png', dpi=300)
plt.show()


# Plot sample Smiles

In [ ]:
import time
for i in range(5):
    sample_ind = i+250
    X_sample = x_test_transform[sample_ind]
    y_sample = y_test[sample_ind]
    print(X_test[sample_ind])
    prediction=modelGEN.predict(X_sample.reshape(1,6))[0]
#     prediction=scale.inverse_transform(modelGEN.predict(X_sample.reshape(1,6))[0])
    plt.figure(1,figsize=(14,6))
    for i in range(maturities_dim):
        plt.subplot(2,4,i+1)

        plt.plot(logmoneyness, y_sample[i*strikes_dim:(i+1)*strikes_dim],'b',label="Input data")
        plt.plot(logmoneyness, prediction[i*strikes_dim:(i+1)*strikes_dim],'--r',label=" NN Approx")


        plt.title("Maturity=%1.2f "%maturities[i])
        plt.xlabel("log-moneyness")
        plt.ylabel("Implied vol")

        plt.legend()
    plt.tight_layout()
    plt.show()
#     time.sleep(5)
    

$\alpha\in[0.505, 0.7],\; \lambda\in[0.5, 1.5],\; a\in[0.1, 0.75],\; b\in[0.05, 0.5],\; c\in[0.0001, 0.01],\; Z_0\in[0.05, 0.3]$

# Use gradient methods for optimisation

In [ ]:
def CostFunc(x,sample_ind):
    return np.sum(np.power((NeuralNetwork(x)-y_test[sample_ind]),2))
def Jacobian(x,sample_ind):
    return 2*np.sum((NeuralNetwork(x)-y_test[sample_ind])*NeuralNetworkGradient(x),axis=1)
#Cost Function for Levenberg Marquardt
def CostFuncLS(x,sample_ind):
    return (NeuralNetwork(x)-y_test[sample_ind])

def JacobianLS(x,sample_ind):
    return NeuralNetworkGradient(x).T

In [ ]:
import time

Approx=[]
Timing = []

solutions=np.zeros(6)
init=lb

for i in range(len(X_test)):
    disp=str(i+1)+"/" + str(len(X_test))
    print (disp, end="\r")
    #Levenberg-Marquardt
    start= time.time()
    I=scipy.optimize.least_squares(CostFuncLS, init, JacobianLS,method='lm',args=(i,),gtol=1E-10)
    end= time.time()
    solutions=myinverse(I.x)
    times=end-start
    
    Approx.append(np.copy(solutions))
    Timing.append(np.copy(times))
    
print("time spent: %ss"%np.round(np.sum(Timing), 6))
# print("solution: ", np.round(np.mean(Approx, axis=0), 6))

# Save the Lavenberg-Marquardt Optimal parameters

In [ ]:
LMParameters=np.array([Approx[i] for i in range(len(Approx))])
np.savetxt("NNParametersQuadraticRoughHeston.txt",LMParameters)
LMParameters = np.loadtxt("NNParametersQuadraticRoughHeston.txt")
LMParameters.shape
np.mean(LMParameters, axis=0)
# Approx[0][0], Approx[0][1]

# Calibration Errors with Levenberg-Marquardt

In [ ]:
titles=["$\\alpha$","$\\lambda$","$a$","$b$", "$c$","$Z_0$"]
average=np.zeros([6,len(X_test)])
fig=plt.figure(figsize=(12,8))
for u in range(6):
    ax=plt.subplot(2,3,u+1)
    x=X_test[:len(X_test),u]
#     print(abs(LMParameters[:1800,u]))
#     print(100*np.abs(LMParameters[:1800,u]))
    plt.plot(x,100*np.abs(LMParameters[:len(X_test),u]-x)/np.abs(x),'b*')
    average[u,:]=np.abs(LMParameters[:len(X_test),u]-x)/np.abs(x)
    
    plt.title(titles[u],fontsize=20)
    plt.ylabel('relative Error',fontsize=15)
    plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter() )
    plt.tick_params(axis='both', which='major', labelsize=15)
    plt.tick_params(axis='both', which='minor', labelsize=15)
    plt.text(0.5, 0.8, 'Average: %1.2f%%\n Median:   %1.2f%% '%(np.mean(100*average[u,:]),np.quantile(100*average[u,:],0.5)), horizontalalignment='center',verticalalignment='center', transform=ax.transAxes,fontsize=15)
plt.tight_layout()

# plt.savefig('qrHestonParameterRelativeErrors.png', dpi=300)
plt.show()

In [ ]:
titles=["$\\alpha$","$\\lambda$","$a$","$b$", "$c$","$Z_0$"]
average=np.zeros([6,len(X_test)])
fig=plt.figure(figsize=(12,8))
for u in range(6):
    ax=plt.subplot(2,3,u+1)
    y=X_test[:len(X_test),u]
    plt.plot(y,np.abs(LMParameters[:len(X_test),u]-y),'b*')
    average[u,:]=np.abs(LMParameters[:len(X_test),u]-y)
    
    plt.title(titles[u],fontsize=20)
    plt.ylabel('Absolute Error',fontsize=15)
    #plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter() )
    plt.tick_params(axis='both', which='major', labelsize=15)
    plt.tick_params(axis='both', which='minor', labelsize=15)
    plt.text(0.5, 0.8, 'Average: %1.2f%%\n Median:   %1.2f%% '%(np.mean(average[u,:]),np.quantile(average[u,:],0.5)), horizontalalignment='center',verticalalignment='center', transform=ax.transAxes,fontsize=15)
plt.tight_layout()

# plt.savefig('qrHestonParameterAbsoluteErrors.png', dpi=300)
plt.show()

In [ ]:
plt.figure(figsize=(18, 5))
plt.clf()
plt.subplot(121)
ax = plt.gca()
q=np.linspace(0,0.99,200)
for u in range(6):
    p=plt.plot(100*q,np.quantile(100*average[u,:],q),label=titles[u])
    
    c=p[0].get_color()
ymin, ymax = ax.get_ylim()
ax.set_xlim(0,100)
plt.plot(100*np.ones(2)*0.95,np.array([0,ymax]),'--k',label="95% quantile")
plt.title("Empirical CDF of parameter relative error",fontsize=20)
plt.legend(fontsize=17)
plt.xlabel("quantiles",fontsize=17)
plt.ylabel("relative error",fontsize=17)
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter() )
plt.gca().xaxis.set_major_formatter(mtick.PercentFormatter() )
plt.tick_params(axis='both', which='major', labelsize=17)
plt.tick_params(axis='both', which='minor', labelsize=17)
plt.xticks(np.arange(0, 101, step=10))
plt.grid()

plt.subplot(122)

ax = plt.gca()
q=np.linspace(0,1,200)
p=plt.plot(100*q,np.quantile(100*RMSE,q),linewidth=3,label="RMSE")
ymin, ymax = ax.get_ylim()
plt.plot(100*np.ones(2)*0.99,np.array([0,ymax]),'--k',label="99% quantile")
plt.title("Empirical CDF of implied vol surface RMSE",fontsize=20)
plt.legend(fontsize=17)
plt.xlabel("quantiles",fontsize=17)
plt.ylabel("RMSE",fontsize=17)
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter() )
plt.gca().xaxis.set_major_formatter(mtick.PercentFormatter() )
plt.tick_params(axis='both', which='major', labelsize=17)
plt.tick_params(axis='both', which='minor', labelsize=17)
plt.xticks(np.arange(0, 101, step=10))
plt.grid()
plt.savefig('qrHestonErrorCDF.png', dpi=300)
plt.show()


## Calibration on market data

In [5]:
from scipy import  interpolate
from scipy.interpolate import griddata
import json

df_logmoneyness_market = json.load(open('logmoneyness_market.json'))
df_market = json.load(open('impliedVols_market.json'))
### maturities and strikes on the market
maturities_market = [0.06, 0.15, 0.31, 0.56]
print("Market logmoneyness: ",df_logmoneyness_market[10])
# print("Model logmoneyness: ",np.round(logmoneyness,3))

Market logmoneyness:  [-0.118, -0.097, -0.077, -0.056, -0.008, 0.039, 0.083, 0.126, 0.167]


In [ ]:
def interpolation_NN(x):
    """
    a function for the interpolation of the market data to the NeuralNetwork
    Input: 
        x: the input sample for the Neural Network function
        logmonyness_market(dim=5), maturities_market(dim=4)
    Output: 
        f: the interpolation results of the NeuralNetwork, 
        df: the interpolation results of the NeuralNetworkGradient
    """
    logKs, Ts   = np.meshgrid(logmoneyness, maturities)
    logKs_, Ts_ = np.meshgrid(logmoneyness_market, maturities_market)
#     points      = np.vstack((logKs.ravel(), Ts.ravel())).T
#     f           = griddata(points, NeuralNetwork(x),(logKs_.ravel(), Ts_.ravel()), method='cubic')
#     sbs = interpolate.SmoothBivariateSpline(logKs.ravel(), Ts.ravel()NNParameters NeuralNetwork(x))
    print(NeuralNetwork(x).shape)

    sbs = interpolate.RectBivariateSpline(maturities, logmoneyness, NeuralNetwork(x).reshape(maturities_dim,-1))

    fnew = sbs.ev(maturities_market, logmoneyness_market)                         
    return fnew

def interpolation_NN_Gradient(x):
    """
    a function for the interpolation of the market data to the NeuralNetworkGradient
    Input: 
        x: the input sample for the Neural Network function
        logmonyness_market(dim=5), maturities_market(dim=4)
    Output: 
        gradient: the interpolation results of the NeuralNetworkGradient
    """
    logKs, Ts   = np.meshgrid(logmoneyness, maturities)
    logKs_, Ts_ = np.meshgrid(logmoneyness_market, maturities_market)
    
    gradient    = np.zeros((6, len(logmoneyness_market)*len(maturities_market)))
    nnGradient  = NeuralNetworkGradient(x)
    
    for i in range(6):
        nnGradient[i].reshape(maturities_dim,-1).shape
        sbs = interpolate.RectBivariateSpline(maturities, logmoneyness, nnGradient[i].reshape(maturities_dim,-1))
        gradient[i] = sbs.ev(maturities_market, logmoneyness_market)  
    
#     for i in range(6):
#         df = interp2d(logKs, Ts, nnGradient[i])
#         gradient[i] = df(logmonyness_market, maturities_market).ravel()
    return gradient
    

# def interpolation_NN_Gradient(x):
#     """
#     a function for the interpolation of the market data to the NeuralNetworkGradient
#     Input: 
#         x: the input sample for the Neural Network function
#         logmonyness_market(dim=5), maturities_market(dim=4)
#     Output: 
#         gradient: the interpolation results of the NeuralNetworkGradient
#     """
#     logKs, Ts   = np.meshgrid(logmoneyness, maturities)
#     logKs_, Ts_ = np.meshgrid(logmoneyness_market, maturities_market)
#     points      = np.vstack((logKs.ravel(), Ts.ravel())).T
#     fnew = interpolate.bisplev(logmoneyness_market, maturities_market)
#     gradient    = np.zeros((6, len(logmoneyness_market)*len(maturities_market)))
#     nnGradient  = NeuralNetworkGradient(x)
    
#     for i in range(6):
#         gradient[i] = griddata(points, nnGradient[i],(logKs_.ravel(), Ts_.ravel()), method='cubic')
    
# #     for i in range(6):
# #         df = interp2d(logKs, Ts, nnGradient[i])
# #         gradient[i] = df(logmonyness_market, maturities_market).ravel()
#     return gradient
    

In [ ]:
interpolation_NN(X_sample[0])

In [ ]:
def CostFunc(x,sample_ind):
    prediction = interpolation_NN(x)
    return np.sum(np.power((prediction - df_market[sample_ind]),2))

def Jacobian(x,sample_ind):
    prediction = interpolation_NN(x)
    gradient = interpolation_NN_Gradient(x)
    
    return 2*np.sum((prediction - df_market[sample_ind])*gradient,axis=1)

# #Cost Function for Levenberg Marquardt

def CostFuncLS(x,sample_ind):
    prediction = interpolation_NN(x)
    return (prediction - df_market[sample_ind])

def JacobianLS(x,sample_ind):
    return interpolation_NN_Gradient(x).T

In [ ]:
bnds = ((-1,1), (-1,1),(-1,1), (-1,1),(-1,1), (-1,1))
type(bnds)
np.max([i[-1] for i in df_logmoneyness_market]), np.log(122/100)

In [ ]:
Approx=[]
Timing = []

solutions=np.zeros([3,6])
init=[(ub[i]+lb[i])*0.5 for i in range(6)]
# init = lb
# init = np.array([0.51, 1.2, 0.384, 0.095, 0.0025, 0.1])
times = np.zeros(3)
for i in range(len(df_market)):
    disp=str(i+1)+"/" + str(len(df_market))
    print (disp, end="\r")
    logmoneyness_market = df_logmoneyness_market[i]
    
    start= time.time()
#     I=scipy.optimize.minimize(CostFunc,x0=init,args=i,method='SLSQP', jac=Jacobian,bounds=bnds, tol=1E-10,options={"maxiter":5000})
    I=scipy.optimize.minimize(CostFunc,x0=init,args=i,method='SLSQP', jac=Jacobian, tol=1E-10,options={"maxiter":5000})
    end= time.time()
    solutions[0,:]=myinverse(I.x)
    times[0]=end-start
    
    start= time.time()
    I=scipy.optimize.minimize(CostFunc,x0=init,args=i,method='L-BFGS-B', jac=Jacobian, tol=1E-10,options={"maxiter":5000})
#     I=scipy.optimize.minimize(CostFunc,x0=init,args=i,method='L-BFGS-B', jac=Jacobian,bounds=bnds, tol=1E-10,options={"maxiter":5000})
    end= time.time()
    solutions[1,:]=myinverse(I.x)
# #     print(solutions[1,:]) 
    times[1]=end-start
    
   # Levenberg-Marquardt
    start= time.time()
    I=scipy.optimize.least_squares(CostFuncLS, x0=init,jac=JacobianLS,method='lm', args=(i,),gtol=1E-10)
#     I=scipy.optimize.least_squares(CostFuncLS, x0=init,jac=JacobianLS,bounds=(-np.ones(6), np.ones(6)), args=(i,),gtol=1E-10)
    end= time.time()
    solutions[2, :]=myinverse(I.x)
    times[2] = end-start
    
    Approx.append(np.copy(solutions))
    Timing.append(np.copy(times))
    
print("time spent: %ss"%np.round(np.sum(Timing, axis=0), 6))
# print("solution: ", np.round(np.mean(Approx, axis=0), 6))

In [ ]:
# for i in range(40):
#     print(np.mean(Approx[i][1]-Approx[i][2]))
np.mean([np.power(Approx[i][0]-Approx[i][2], 2) for i in range(len(Approx))])

In [ ]:
### results with bounds
for j in range(3):
    tmp = [Approx[i][j] for i in range(len(Approx))]
    print("Mean:",np.mean(tmp, axis=0))
    print("Std: ",np.round(np.std(tmp, axis=0), 10))
    print("-------------------------------------------------------------------------------")
    